In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import cv2
import math
import numpy as np
from skimage import measure
from skimage.measure import regionprops
from skimage.morphology import opening, erosion, dilation, disk
import matplotlib.pyplot as plt

In [ ]:
PRED_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/ER/Combined FTL/"
# PRED_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/ER/3Fold HScoreNet/"
GT_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/4 Class ER/Masks/"

In [ ]:
# Axis 0 = Fold
# Axis 1 = Class (Strong Intermediate Weak)
# Axis 2 = Metric (Precision Recall Dice)
objEvaluationTrain = np.empty((0,3,3))
objEvaluationTest = np.empty((0,3,3))
score = np.zeros((1,3,3))

In [ ]:
FOLD_PATH = []
files = os.listdir(PRED_PATH)

# Removing all the .np files from list of folds
for i in range(len(files)):
  if(files[i].split('.')[-1]=="npy"):
    continue
  FOLD_PATH.append(files[i])
print(FOLD_PATH)

['Fold 0', 'Fold 1']


In [ ]:
def updateConfusionMatrix(ConfusionMatrix, PredictedImage, GroundTruth, axis):
    # Opening operation
    selem = disk(15)
    GroundTruth = dilation(GroundTruth,selem)
    GroundTruthLabel = measure.label(GroundTruth)

    rows,cols = PredictedImage.shape

    #  0 is background
    for label in range(1, GroundTruthLabel.max()+1):
      masklabel = np.zeros([rows,cols])
      masklabel[GroundTruthLabel==label] = 1

      resultantimage = cv2.multiply(PredictedImage, masklabel)

      areaGT = np.sum(masklabel)
      areaSM = np.sum(resultantimage)

      if(areaSM > 0 ):
        ConfusionMatrix[axis][0] = ConfusionMatrix[axis][0] + 1       #TP
        PredictedImage[GroundTruthLabel==label] = 0
      else:
        ConfusionMatrix[axis][2] = ConfusionMatrix[axis][2] + 1       #FN
    
    predictedprops = regionprops(measure.label(PredictedImage))

    for i in range(len(predictedprops)):
      if(predictedprops[i].area >= 2000):
        ConfusionMatrix[axis][1] += 1                                #FP

In [ ]:
def getPatientWiseCount(ConfusionMatrix, PredictedPatient, GroundTruthPatient):

  for imagenum in range(PredictedPatient.shape[0]):
  
    PredictedImage = PredictedPatient[imagenum]
    GroundTruth    = GroundTruthPatient[imagenum]
    rows,cols = PredictedImage.shape

    # dividing the image into strong,weak,intermediate
    predictedStrong = np.zeros([rows,cols])
    predictedWeak   = np.zeros([rows,cols])
    predictedIntermediate = np.zeros([rows,cols])

    gtStrong = np.zeros([rows,cols])
    gtWeak = np.zeros([rows,cols])
    gtIntermediate = np.zeros([rows,cols])

    predictedStrong[PredictedImage==255] = 1
    predictedIntermediate [PredictedImage==170] = 1
    predictedWeak[PredictedImage==85] = 1

    gtStrong[GroundTruth==255] = 1
    gtIntermediate[GroundTruth==170] = 1
    gtWeak[GroundTruth==85] = 1

    # 0th axis in confusion matrix is strong 
    # 1st axis in confusion matrix is intermediate 
    # 2nd axis in confusion matrix is weak
    updateConfusionMatrix(ConfusionMatrix, predictedStrong, gtStrong, 0)
    updateConfusionMatrix(ConfusionMatrix, predictedIntermediate, gtIntermediate, 1)
    updateConfusionMatrix(ConfusionMatrix, predictedWeak, gtWeak, 2)

In [ ]:
def getMetrics(m, axis):
  precision = 1
  recall = 1
  dice = 1
  if(m[axis][0] + m[axis][1] != 0):
    precision = m[axis][0] / (m[axis][0] + m[axis][1])

  if(m[axis][0] + m[axis][2] != 0):
    recall    = m[axis][0] / (m[axis][0] + m[axis][2])
    
  if(precision + recall != 0):
    dice      = (2 * precision * recall) / (precision + recall)
  else:
    dice = 0

  return [precision, recall, dice]

In [ ]:
for fold in range(len(FOLD_PATH)):

  print("Fold: " + str(fold))
  
  train_path = os.listdir(PRED_PATH + FOLD_PATH[fold] + '/Train/')
  test_path = os.listdir(PRED_PATH + FOLD_PATH[fold] + '/Test/')

  train_confusion_matrix = np.zeros([ 3, 3])
  test_confusion_matrix  = np.zeros([ 3, 3])

  # TRAIN
  for patientID in train_path:    
    SM = np.load(PRED_PATH + FOLD_PATH[fold] + '/Train/' + patientID)
    GT = np.load(GT_PATH + 'IHC ' + patientID)
    getPatientWiseCount(train_confusion_matrix, SM, GT)

  for i in range(3):
    metrics = getMetrics(train_confusion_matrix, i)
    for j in range(len(metrics)):
      score[0][i][j] = metrics[j]
    
  objEvaluationTrain = np.append(objEvaluationTrain,score,axis=0)
  print(score)

  # TEST
  for patientID in test_path:
    SM = np.load(PRED_PATH + FOLD_PATH[fold] + '/Test/' + patientID)
    GT = np.load(GT_PATH + 'IHC ' + patientID)
    getPatientWiseCount(test_confusion_matrix, SM, GT)

  for i in range(3):
    metrics = getMetrics(test_confusion_matrix, i)
    for j in range(len(metrics)):
      score[0][i][j] = metrics[j]
  
  objEvaluationTest = np.append(objEvaluationTest,score,axis=0)
  print(score)

Fold: 0
[[[0.72320377 0.97460317 0.83029074]
  [0.56929348 0.93526786 0.70777027]
  [0.6875     0.53921569 0.6043956 ]]]
[[[0.98127341 0.74643875 0.84789644]
  [0.4884106  0.98662207 0.65337763]
  [0.82926829 0.408      0.54691689]]]
Fold: 1
[[[0.81981982 0.92857143 0.8708134 ]
  [0.63732929 0.92105263 0.75336323]
  [0.57504873 0.81043956 0.672748  ]]]
[[[0.67244367 0.98727735 0.8       ]
  [0.88501742 0.87285223 0.87889273]
  [0.42010309 0.84895833 0.56206897]]]


In [ ]:
print(objEvaluationTrain.shape)
print(objEvaluationTest.shape)

(2, 3, 3)
(2, 3, 3)


In [ ]:
SAVE_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/ER/Combined FTL/"
# SAVE_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/ER/3Fold HScoreNet/"
np.save(SAVE_PATH + "Object-wise Metrics Train.npy", objEvaluationTrain)
np.save(SAVE_PATH + "Object-wise Metrics Test.npy", objEvaluationTest)

In [5]:
#To calculate average, precision and recall
# LOAD_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/ER/Combined FTL/"
LOAD_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/HER2/3Fold HER2NET/"
trainMetrics = np.load(LOAD_PATH + "Pixel-wise Metrics Train.npy")
testMetrics  = np.load(LOAD_PATH + "Pixel-wise Metrics Test.npy")
print(trainMetrics.shape)
print(testMetrics.shape)

(3, 3, 3)
(3, 3, 3)


In [6]:
print(trainMetrics)

[[[1.         0.         0.        ]
  [1.         0.         0.        ]
  [0.65194878 1.         0.78930871]]

 [[1.         0.         0.        ]
  [1.         0.         0.        ]
  [0.65734167 1.         0.79324823]]

 [[1.         0.         0.        ]
  [1.         0.         0.        ]
  [0.66807315 1.         0.80101181]]]


In [7]:
print(testMetrics)

[[[1.         0.         0.        ]
  [1.         0.         0.        ]
  [0.67794068 1.         0.80806275]]

 [[1.         0.         0.        ]
  [1.         0.         0.        ]
  [0.65872666 1.         0.79425583]]

 [[1.         0.         0.        ]
  [1.         0.         0.        ]
  [0.62069918 1.         0.7659647 ]]]


In [8]:
#For pixel-wise background is included, object-wise background is not included
numClasses = 3
avgTrainMetrics  = np.zeros([numClasses, 3])
avgTestMetrics   = np.zeros([numClasses, 3])

def calculateMetrics(AvgMetrics, ValidationMetrics):
  for patientNum in range(ValidationMetrics.shape[0]):
    for row in range(numClasses):
      for col in range(3):
        AvgMetrics[row][col] += ValidationMetrics[patientNum][row][col]
  AvgMetrics = np.divide(AvgMetrics,ValidationMetrics.shape[0] )
  return AvgMetrics


avgTrainMetrics = calculateMetrics(avgTrainMetrics, trainMetrics)
print('TRAIN :')
print(avgTrainMetrics)

avgTestMetrics = calculateMetrics(avgTestMetrics, testMetrics)
print('TEST :')
print(avgTestMetrics)

TRAIN :
[[1.         0.         0.        ]
 [1.         0.         0.        ]
 [0.6591212  1.         0.79452292]]
TEST :
[[1.         0.         0.        ]
 [1.         0.         0.        ]
 [0.65245551 1.         0.78942776]]


# **SM = Segmented Mask**

**GT = Ground Truth** 

In [ ]:
train_confusion_matrix = np.zeros([ 3, 3])
test_confusion_matrix  = np.zeros([ 3, 3])

for patientid in range(len(TRAIN)):
  patientSM = np.load(TRAIN_PATH + TRAIN[patientid] + '.npy')
  patientGT      = np.load(MASK_PATH + 'IHC ' + TRAIN[patientid] + '.npy')
  getPatientWiseCount(train_confusion_matrix, patientSM, patientGT, patientid)
  print(TRAIN[patientid] + ' done.')
  print(train_confusion_matrix)

232 done.
[[277.  14.  74.]
 [276. 161.  23.]
 [151.  96.  26.]]
263 done.
[[491.  72.  97.]
 [425. 302.  31.]
 [225. 176.  59.]]
230 done.
[[491.  72.  97.]
 [425. 302.  31.]
 [277. 178.  80.]]
242 done.
[[491.  72.  97.]
 [425. 302.  31.]
 [277. 178.  80.]]


In [ ]:
for patientid in range(len(TEST)):
  patientSM = np.load(TEST_PATH + TEST[patientid] + '.npy')
  patientGT      = np.load(MASK_PATH + 'IHC ' + TEST[patientid] + '.npy')
  getPatientWiseCount(test_confusion_matrix, patientSM, patientGT, patientid)
  print(TEST[patientid] + ' done.')
  print(test_confusion_matrix)

239 done.
[[384. 185.   9.]
 [241.  65.  50.]
 [143. 133.  49.]]
221 done.
[[384. 185.   9.]
 [241.  65.  50.]
 [144. 133.  49.]]
229 done.
[[384. 185.   9.]
 [241.  65.  50.]
 [144. 133.  49.]]
246 done.
[[384. 185.   9.]
 [241.  65.  50.]
 [150. 133.  49.]]
252 done.
[[384. 185.   9.]
 [241.  66.  50.]
 [150. 134.  49.]]


In [ ]:
strain_precision, strain_recall, strain_dice = getMetrics(train_confusion_matrix, 0)
itrain_precision, itrain_recall, itrain_dice = getMetrics(train_confusion_matrix, 1)
wtrain_precision, wtrain_recall, wtrain_dice = getMetrics(train_confusion_matrix, 2)

print('TRAIN :\n\n')
print("CLASS\t\tPRECISION\t\tRECALL\t\t\tDICE")
print('Strong\t\t' + str(strain_precision) + '\t' + str(strain_recall) + '\t' + str(strain_dice))
print('Intermediate\t' + str(itrain_precision) +  '\t' + str(itrain_recall) + '\t' + str(itrain_dice))
print('Weak\t\t' + str(wtrain_precision) +   '\t' + str(wtrain_recall) + '\t' + str(wtrain_dice))

TRAIN :


CLASS		PRECISION		RECALL			DICE
Strong		0.872113676731794	0.8350340136054422	0.8482026186792665
Intermediate	0.5845942228335625	0.9320175438596491	0.7138056755583373
Weak		0.6087912087912087	0.7759103641456583	0.677374167589391


In [ ]:
stest_precision, stest_recall, stest_dice = getMetrics(test_confusion_matrix, 0)
itest_precision, itest_recall, itest_dice = getMetrics(test_confusion_matrix, 1)
wtest_precision, wtest_recall, wtest_dice = getMetrics(test_confusion_matrix, 2)

print('TEST :\n\n')
print("CLASS\t\tPRECISION\t\tRECALL\t\t\tDICE")
print('Strong\t\t' + str(stest_precision) + '\t' + str(stest_recall) + '\t' + str(stest_dice))
print('Intermediate\t' + str(itest_precision) +  '\t' + str(itest_recall) + '\t' + str(itest_dice))
print('Weak\t\t' + str(wtest_precision) +   '\t' + str(wtest_recall) + '\t' + str(wtest_dice))

TEST :


CLASS		PRECISION		RECALL			DICE
Strong		0.6748681898066784	0.9770992366412213	0.7935332336842523
Intermediate	0.7850162866449512	0.8281786941580757	0.8010544276631549
Weak		0.528169014084507	0.7537688442211056	0.6163103662307937
